# UK Tissue Detection
Investigate the annotations created by the UK(entucky) group and now stored on an Emory-based instance of the DSA.

In [ ]:
# Imports
from tqdm.notebook import tqdm
import numpy as np
from pathlib import Path
from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt

from dsa_helpers.girder_utils import login
from dsa_helpers import imread
from dsa_helpers.ml.tiling import semantic_seg_tile_wsi_with_dsa_annotations

In [ ]:
# Constants
DSA_API_URL = "http://bdsa.pathology.emory.edu:8080/api/v1"
DSA_FLD_ID = "673b566f900c0c0559bf156f"  # folder with WSI items
DSA_DOC_NAME = "Gray White Segmentation"
WSI_DIR = "/wsi_archive/UK_Slides"
TILE_DIR = "/home/jvizcar/data/uk-slides/tiles"
NPROC = 20

label2idx = {
    "Gray Matter": 1,
    'White Matter': 2,
    'Superficial': 3,
    "Leptomeninges": 4
}

color_map = {
    1: [0,128,0],
    2: [0,0,255],
    3: [255,255, 255],
    4: [0,0,0]
}

In [ ]:
# Authenticate DSA instance.
gc = login(DSA_API_URL)

In [ ]:
# Track item metadata.
items = []

# List the items in the DSA folder.
for item in tqdm(list(gc.listItem(DSA_FLD_ID)), desc="Getting item metadata..."):
    ann_docs = []  # list of annotation documents of name: "Gray White Segmentation"
    
    for ann_doc in gc.get(f"annotation/item/{item['_id']}"):
        if ann_doc.get('annotation', {}).get("name") == DSA_DOC_NAME:
            ann_docs.append(ann_doc)
    
    # Include this item only if it has annotation documents.
    if len(ann_docs):
        item['annotation_documents'] = ann_docs
        
        items.append(item)
        break
        
print(f"Found {len(items)} items with annotation documents.")

In [ ]:
# Get the first item to test.
item = items[0]
annotation_docs = item["annotation_documents"]
wsi_fp = str(Path(WSI_DIR) / f"{item['name']}")

# Tile image with semantic segmentation labels.
semantic_seg_tile_wsi_with_dsa_annotations(
    wsi_fp, annotation_docs, label2idx, TILE_DIR, 1024, nproc=NPROC, mag=20
)

In [ ]:
# List all the tile images.
img_dir = Path(TILE_DIR) / "images"
fps = list(img_dir.glob("*.png"))

def plot_images(idx):
    fp = fps[idx]

    img = imread(fp)
    mask = imread(Path(TILE_DIR) / "masks" / fp.name, grayscale=True)
    
    # Remap the mask to the color map.
    mask_rgb = np.zeros(img.shape, dtype=np.uint8)
    
    for idx, color in color_map.items():
        mask_rgb[mask == idx] = color
        
    # Plot them side by side.
    fig, axs = plt.subplots(1, 2, figsize=(15, 15))
    axs[0].imshow(img)
    axs[1].imshow(mask_rgb)
    plt.show()
    

_ = interact(
    plot_images,
    idx=IntSlider(min=0, max=len(fps) - 1, step=1, value=0, continuour_update=False)
)